<a href="https://colab.research.google.com/github/Daithi333/colab-notebooks/blob/main/query_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers

In [ ]:
!pip install chromadb

In [ ]:
!pip install openai
!pip install tiktoken

In [ ]:
import os
import sys

import torch
from langchain.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

## Load Documents and extract text

In [ ]:
!mkdir docs

mkdir: cannot create directory ‘docs’: File exists


In [ ]:
document = []

for file in os.listdir("docs"):
  path = f"./docs/{file}"
  if file.endswith(".pdf"):
    loader = PyPDFLoader(path)
  elif file.endswith(".docx") or file.endswith(".doc"):
    loader = Docx2txtLoader(path)
  elif file.endswith(".docx") or file.endswith(".doc"):
    loader = TextLoader(path)
  else:
    raise ValueError(f"Unrecognised extension on {file}")

  document.extend(loader.load())

In [ ]:
document

[Document(page_content=' Architecting for the Cloud  \nAWS Best Practices  \nOctober  2018 \n \n \n \n \n \n \n \n  \n', metadata={'source': './docs/AWS_Cloud_Best_Practices.pdf', 'page': 0}),
 Document(page_content=' Notices  \nThis document is provided for informational purposes only. It represents AWS’s \ncurrent product offerings and practices as of the date of issue of this document, which \nare subject to change without notice. Customers are responsible for making their own \nindependent assessment of the information in this document and any use of AWS’s \nproducts or services, each of which is provided “as is” without warranty of  any kind, \nwhether express or implied. This document does not create any warranties, \nrepresentations, contractual commitments, conditions or assurances from AWS, its \naffiliates, suppliers or licensors. The responsibilities and liabilities of AWS to its \ncustomers are controlled by AWS agreements, and this document is not part of, nor \ndoes it mo

In [ ]:
len(document)

67

## Split Document into chunks

In [ ]:
document_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
document_chunks = document_splitter.split_documents(document)
len(document_chunks)

67

In [ ]:
document_chunks[0]

Document(page_content='Architecting for the Cloud  \nAWS Best Practices  \nOctober  2018', metadata={'source': './docs/AWS_Cloud_Best_Practices.pdf', 'page': 0})

## Download Embeddings

In [ ]:
# Cost associated with using OpenAI
# os.environ["OPENAI_API_KEY"] = "abc123"
# embeddings = OpenAIEmbeddings()

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

## Set up Chroma Vector Db

In [ ]:
vectordb = Chroma.from_documents(document_chunks, embedding=embeddings, persist_directory="./data")

In [ ]:
vectordb.persist()

## Login to HuggingFace Hub and download LLM

In [ ]:
notebook_login()

## Download Llama 2 7B LLM

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(
#     "meta-llama/Llama-2-7b-chat-hf",
#     use_auth_token=True,
#     )

# model = AutoModelForCausalLM.from_pretrained(
#     "meta-llama/Llama-2-7b-chat-hf",
#     use_auth_token=True,
#     torch_dtype=torch.float16,
#     # load_in_8bit=True,
#     load_in_4bit=True,
# )